In [6]:
import SimpleITK as sitk
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed
from IPython.display import clear_output
import pprint
import os, sys
import pandas_profiling
import sys
import json
from scipy import ndimage
import nibabel as nib 
from pathlib import Path
sys.path.insert(0, str(Path(r'C:\Users\parkm\Desktop\github\analysis_nifti_for_deeplearning\src\registration_metric.pyy').resolve().parent.parent))
from src.eda_nifti import *
from src.registration_metric import *

In [8]:
nifti_anal =NiftiAnalysis()

In [9]:
task1_nifti_files_list = nifti_anal.recursive_find_all_files(r'C:\Users\parkm\Desktop\github\analysis_nifti_for_deeplearning\data\task1', '.nii.gz')

In [3]:
input = sitk.ReadImage(files[0])
image = sitk.Cast(input, sitk.sitkFloat32)
# print(type(image))


correctedImg = sitk.N4BiasFieldCorrection(image)
sitk.WriteImage(correctedImg, outfile) # outfile is just string variable or path

NameError: name 'files' is not defined

In [ ]:
bias_field_full_resolution = sitk.Exp(corrector.GetLogBiasFieldAsImage(biased_image_full_resolution))

In [ ]:
bias_field_full_resolution = sitk.Exp(corrector.GetLogBiasFieldAsImage(biased_image_full_resolution))
corrected_image_full_resolution = biased_image_full_resolution / bias_field_full_resolution